In [1]:
# Instalação das Bibliotecas

# pip install selenium
# pip install tinydb
# pip install fake_useragent

In [2]:
import pandas as pd
from time import sleep
from datetime import date, datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from tqdm.auto import tqdm
from tinydb import TinyDB,Query
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
links1 = ['https://www.flashscore.com/football/ecuador/liga-pro-2021/results/',#1
'https://www.flashscore.com/football/ecuador/liga-pro-2022/results/',
'https://www.flashscore.com/football/ecuador/liga-pro/results/',
'https://www.flashscore.com/football/brazil/brasileiro-women-2021/results/',#2
'https://www.flashscore.com/football/brazil/brasileiro-women-2022/results/',
'https://www.flashscore.com/football/brazil/brasileiro-women/results/',
'https://www.flashscore.com/football/peru/liga-1-2021/results/',#3
'https://www.flashscore.com/football/peru/liga-1-2022/results/',
'https://www.flashscore.com/football/peru/liga-1/results/',
'https://www.flashscore.com/football/bolivia/division-profesional-2021/results/',#4
'https://www.flashscore.com/football/bolivia/division-profesional-2022/results/',
'https://www.flashscore.com/football/bolivia/division-profesional/results/',
'https://www.flashscore.com/football/argentina/torneo-federal-2021/results/',#5
'https://www.flashscore.com/football/argentina/torneo-federal-2022/results/',
'https://www.flashscore.com/football/argentina/torneo-federal/results/']

In [4]:
links2 = [
'https://www.flashscore.com/football/bosnia-and-herzegovina/premijer-liga-bih-2021-2022/results/',#1
'https://www.flashscore.com/football/bosnia-and-herzegovina/premijer-liga-bih-2022-2023/results/',
'https://www.flashscore.com/football/bosnia-and-herzegovina/premijer-liga-bih/results/',
'https://www.flashscore.com/football/ghana/premier-league-2021-2022/results/',#2
'https://www.flashscore.com/football/ghana/premier-league-2022-2023/results/',
'https://www.flashscore.com/football/ghana/premier-league/results/',
'https://www.flashscore.com/football/north-macedonia/1-mfl-2021-2022/results/',#3
'https://www.flashscore.com/football/north-macedonia/1-mfl-2022-2023/results/',
'https://www.flashscore.com/football/north-macedonia/1-mfl/results/',
'https://www.flashscore.com/football/algeria/u21-league-2021-2022/results/',#4
'https://www.flashscore.com/football/algeria/u21-league-2022-2023/results/',
'https://www.flashscore.com/football/algeria/u21-league/results/',
'https://www.flashscore.com/football/switzerland/super-league-2021-2022/results/',#5
'https://www.flashscore.com/football/switzerland/super-league-2022-2023/results/',
'https://www.flashscore.com/football/switzerland/super-league/results/']

In [5]:
def get_main_header(id_jogo,dictionary,driver):
    driver.get(f'https://www.flashscore.com/match/{id_jogo}/#/match-summary/match-summary')
    dictionary['Id'] = id_jogo
    country = driver.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country').text.split(':')[0]
    date = driver.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[0]
    dictionary['Date'] = date.replace('.','/')
    time = driver.find_element(By.CSS_SELECTOR,'div.duelParticipant__startTime').text.split(' ')[1]
    dictionary['Time'] = time
    league = driver.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country > a').text.split(' -')[0]
    dictionary['League'] = f'{country} - {league}'
    home = driver.find_element(By.CSS_SELECTOR,'div.duelParticipant__home').find_element(By.CSS_SELECTOR,'div.participant__participantName').text
    dictionary['Home'] = home
    away = driver.find_element(By.CSS_SELECTOR,'div.duelParticipant__away').find_element(By.CSS_SELECTOR,'div.participant__participantName').text
    dictionary['Away'] = away
    try:
        rodada = driver.find_element(By.CSS_SELECTOR,'span.tournamentHeader__country > a').text.split('- ')[1]
        dictionary['Round'] = rodada
    except:
        dictionary['Round'] = '-'

In [6]:
def get_odds_ml_ft(id_jogo,dictionary,driver):
    url_ml_full_time = f'https://www.flashscore.com/match/{id_jogo}/#/odds-comparison/1x2-odds/full-time'
    driver.get(url_ml_full_time)
    sleep(1)
    if driver.current_url == url_ml_full_time:
        WebDriverWait(driver, 8).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'div.ui-table')))
        table_odds = driver.find_element(By.CSS_SELECTOR,'div.ui-table')
        linha_ml_ft = table_odds.find_element(By.CSS_SELECTOR,'div.ui-table__row')
        dictionary['Odd_H'] = float(linha_ml_ft.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[0].text)
        dictionary['Odd_D'] = float(linha_ml_ft.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[1].text)
        dictionary['Odd_A'] = float(linha_ml_ft.find_elements(By.CSS_SELECTOR,'a.oddsCell__odd')[2].text)

In [7]:
def insert_season(dictionary,season):
    dictionary['Season'] = season

In [8]:
def get_score_ht_ft(id_jogo,dictionary,driver):
    try:
        if '1st Half' in driver.page_source:
            WebDriverWait(driver, 8).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'div.smv__incidentsHeader')))
            home_goals_ht = int(driver.find_elements(By.CSS_SELECTOR,'div.smv__incidentsHeader div')[1].text.split('-')[0])
            dictionary['HT_GH'] = home_goals_ht
            away_goals_ht = int(driver.find_elements(By.CSS_SELECTOR,'div.smv__incidentsHeader div')[1].text.split('-')[1])
            dictionary['HT_GA'] = away_goals_ht
    except:
        pass
    home_goals_ft = driver.find_element(By.CSS_SELECTOR,'div.detailScore__wrapper').text.split('-')[0]
    dictionary['GH'] = int(home_goals_ft)
    away_goals_ft = driver.find_element(By.CSS_SELECTOR,'div.detailScore__wrapper').text.split('-')[1]
    dictionary['GA'] = int(away_goals_ft)

In [9]:
for i in links1:
    
    url = i
    colecao_partidas = TinyDB('./base_flashscore_inverno_bk_casa.json')
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')

    s = Service('./chromedriver.exe')
    driver = webdriver.Chrome(service=s, options=options)
    driver.maximize_window()
    
    try:
        driver.get(url)

        if ('No match found.' in driver.page_source) == False:
            try:
                WebDriverWait(driver, 8).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')))
                button_cookies = driver.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
                button_cookies.click()
            except:
                print("cookies already closed")

            while('event__more' in driver.page_source):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
                button_more = driver.find_element(By.CSS_SELECTOR,'a.event__more.event__more--static')
                button_more.click()
                sleep(2)

            jogos = driver.find_elements(By.CSS_SELECTOR,'div.event__match--twoLine')
            id_jogos =[]
            for i in jogos:
                id_jogos.append(i.get_attribute("id")[4:])

            season = driver.find_element(By.CSS_SELECTOR,'div.heading__info').text

            print(driver.title)
            for id_jogo in tqdm(id_jogos):
                procura_duplicado = Query()
                if colecao_partidas.search(procura_duplicado.Id == id_jogo) == []:
                    try:
                        jogo = {}

                        get_main_header(id_jogo,jogo,driver)
                        insert_season(jogo,season)
                        get_score_ht_ft(id_jogo,jogo,driver)
                        get_odds_ml_ft(id_jogo,jogo,driver)
                        
                       

                        colecao_partidas.insert(jogo.copy())
                    except:
                        pass
        driver.close()
    except:
        pass

Liga Pro 2021 Results - Football/Ecuador


  0%|          | 0/242 [00:00<?, ?it/s]

Liga Pro 2022 Results - Football/Ecuador


  0%|          | 0/242 [00:00<?, ?it/s]

Liga Pro 2023 Results - Football/Ecuador


...

  0%|          | 0/465 [00:00<?, ?it/s]

Torneo Federal 2022 Results - Football/Argentina


  0%|          | 0/559 [00:00<?, ?it/s]

Torneo Federal 2023 Results - Football/Argentina


  0%|          | 0/593 [00:00<?, ?it/s]

In [10]:
for i in links2:
    
    url = i
    colecao_partidas = TinyDB('./base_flashscore_verão_bk_casa.json')
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')

    s = Service('./chromedriver.exe')
    driver = webdriver.Chrome(service=s, options=options)
    driver.maximize_window()

    
    try:
        driver.get(url)

        if ('No match found.' in driver.page_source) == False:
            try:
                WebDriverWait(driver, 8).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')))
                button_cookies = driver.find_element(By.CSS_SELECTOR,'button#onetrust-accept-btn-handler')
                button_cookies.click()
            except:
                print("cookies already closed")

            while('event__more' in driver.page_source):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
                button_more = driver.find_element(By.CSS_SELECTOR,'a.event__more.event__more--static')
                button_more.click()
                sleep(2)

            jogos = driver.find_elements(By.CSS_SELECTOR,'div.event__match--twoLine')
            id_jogos =[]
            for i in jogos:
                id_jogos.append(i.get_attribute("id")[4:])

            season = driver.find_element(By.CSS_SELECTOR,'div.heading__info').text

            print(driver.title)
            for id_jogo in tqdm(id_jogos):
                procura_duplicado = Query()
                if colecao_partidas.search(procura_duplicado.Id == id_jogo) == []:
                    try:
                        jogo = {}

                        get_main_header(id_jogo,jogo,driver)
                        insert_season(jogo,season)
                        get_score_ht_ft(id_jogo,jogo,driver)
                        get_odds_ml_ft(id_jogo,jogo,driver)
                   
                       

                        colecao_partidas.insert(jogo.copy())
                    except:
                        pass
        driver.close()
    except:
        pass

Premijer liga BiH 2021/2022 Results - Football/Bosnia and Herzegovina


  0%|          | 0/198 [00:00<?, ?it/s]

Premijer liga BiH 2022/2023 Results - Football/Bosnia and Herzegovina


  0%|          | 0/198 [00:00<?, ?it/s]

Premijer liga Results - Football/Bosnia and Herzegovina


...

  0%|          | 0/182 [00:00<?, ?it/s]

Super League 2022/2023 Results - Football/Switzerland


  0%|          | 0/182 [00:00<?, ?it/s]

Super League 2023/2024 Results - Football/Switzerland


  0%|          | 0/108 [00:00<?, ?it/s]

In [11]:
colecao_partidas1 = TinyDB('./base_flashscore_inverno_bk_casa.json')
colecao_partidas2 = TinyDB('./base_flashscore_verão_bk_casa.json')

df1 = pd.DataFrame(colecao_partidas1.all())
df2 = pd.DataFrame(colecao_partidas2.all())

df = pd.concat([df1,df2])
df.drop_duplicates(subset=['Id'], inplace=True)

df[['Dia','Mes','Ano']] = df['Date'].str.split('/',expand=True)
df = df.drop(['Date'], axis=1)
df['Date'] = df['Dia']+'-'+df['Mes']+'-'+ df['Ano']
df = df.drop(columns=['Dia','Mes','Ano'], axis=1)

df = df[['Date','Time','League','Season','Round','Home','Away','GH','GA',
'Odd_H','Odd_D','Odd_A']]

df.columns = ['Date','Time','League','Season','Round','Home','Away','GH','GA',
'Odd_H','Odd_D','Odd_A']

df.fillna(0, inplace=True)

df = df.sort_values(['Date','Time'])

df = df.dropna()
df = df.reset_index(drop=True)
df.index += 1

df.to_excel('Base_de_Dados_FlashScore_1x2_bk_casa.xlsx')